In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import regex as re

In [2]:
pd.set_option('max_columns', None)

In [153]:
df_laerm = pd.read_csv('daten/df_sg_FINAL.csv')

## 1. Daten bereinigen

In [164]:
## zur Sicherheit die NaN entfernen, falls es noch welche hat


cols = ["Emissionswert Tag","Emissionswert Nacht"]
df_laerm[cols] = df_laerm[cols].replace({'0':np.nan, 0:np.nan})


df_laerm.dropna(subset=["Emissionswert Tag","Emissionswert Nacht"], inplace = True)

### Daten zur Autobahn löschen

Die Angaben sind zwar interessant, tragen aber wenig zur Auswertung bei. Auf der Autobahn gehen die Leute ja nicht spazieren, sollten sie zumindest nicht ...

In [190]:
df_laerm = df_laerm[df_laerm['strasse'] != 'A1'].copy()
df_laerm  = df_laerm[df_laerm['strasse'] != '447 33'].copy()

In [191]:
df_laerm.sort_values('strasse', ascending = True)

,GPLZ,gemeinde,Geo Shape PLZ,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
10407,9430,St.Margrethen SG,"{""coordinates"": [[[9.6297385914, 47.4398393524...",MULTILINESTRING Z ((1258621.216899998 2764020....,Laermemission.8146,8146,KS,77,67,9600.0,...,NaN,NaN,NaN,1258603.897 2764050.919 0,"(47.45774705088517, 9.614425302851588)","13 10, 9430 St.Margrethen, Switzerland",13 10,9430 St.Margrethen,9430,St.Margrethen
7568,9450,Altstätten SG,"{""coordinates"": [[[9.5405633213, 47.3394737085...",MULTILINESTRING Z ((1250600.749000002 2759722....,Laermemission.4361,4361,KS,77,69,10500.0,...,NaN,NaN,NaN,1250605.475000001 2759730.464000002 0,"(47.38689812188921, 9.554291163758604)","13 101, 9450 Altstätten, Switzerland",13 101,9450 Altstätten,9450,Altstätten
7643,9450,Lüchingen,"{""coordinates"": [[[9.5827122973, 47.377249934]...",MULTILINESTRING Z ((1250600.749000002 2759722....,Laermemission.4361,4361,KS,77,69,10500.0,...,NaN,NaN,NaN,1250605.475000001 2759730.464000002 0,"(47.38689812188921, 9.554291163758604)","13 101, 9450 Altstätten, Switzerland",13 101,9450 Altstätten,9450,Altstätten
7793,9450,Altstätten SG 2,NaN,MULTILINESTRING Z ((1250600.749000002 2759722....,Laermemission.4361,4361,KS,77,69,10500.0,...,NaN,NaN,NaN,1250605.475000001 2759730.464000002 0,"(47.38689812188921, 9.554291163758604)","13 101, 9450 Altstätten, Switzerland",13 101,9450 Altstätten,9450,Altstätten
7718,9450,Altstätten SG Zust,NaN,MULTILINESTRING Z ((1250600.749000002 2759722....,Laermemission.4361,4361,KS,77,69,10500.0,...,NaN,NaN,NaN,1250605.475000001 2759730.464000002 0,"(47.38689812188921, 9.554291163758604)","13 101, 9450 Altstätten, Switzerland",13 101,9450 Altstätten,9450,Altstätten
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8486,9534,Gähwil,"{""coordinates"": [[[8.9714793354, 47.3912879722...",MULTILINESTRING Z ((1252330.943999998 2717411....,Laermemission.1139,1139,GS,66,47,1000.0,...,NaN,NaN,NaN,1252316.395 2717418.728999998 0,"(47.411187035277024, 8.994485782094578)","Ötwil 2944, 9534 Gähwil, Switzerland",Ötwil 2944,9534 Gähwil,9534,Gähwil
8503,9534,Gähwil,"{""coordinates"": [[[8.9714793354, 47.3912879722...",MULTILINESTRING Z ((1252417.191 2717368.879500...,Laermemission.3590,3590,GS,70,52,1000.0,...,NaN,NaN,NaN,1252405.925999999 2717376.153999999 0,"(47.4119997240904, 8.993945304294392)","Ötwil 2944, 9534 Gähwil, Switzerland",Ötwil 2944,9534 Gähwil,9534,Gähwil
8480,9534,Gähwil,"{""coordinates"": [[[8.9714793354, 47.3912879722...",MULTILINESTRING Z ((1251048.873599999 2717869....,Laermemission.594,594,GS,69,50,1000.0,...,NaN,NaN,NaN,1251040.357000001 2717889.370000001 0,"(47.399628304929834, 9.000383836174175)","Ötwilerstrasse 10, 9534 Gähwil, Switzerland",Ötwilerstrasse 10,9534 Gähwil,9534,Gähwil
4228,9512,Rossrüti,"{""coordinates"": [[[9.0641119484, 47.4683020977...",MULTILINESTRING Z ((1261027.407000002 2722150....,Laermemission.4832,4832,GS,75,61,2500.0,...,NaN,NaN,NaN,1261022.546 2722153.993000001 0,"(47.48861216395646, 9.059596310744595)","Ürentalerstrasse 20, 9512 Rossrüti, Switzerland",Ürentalerstrasse 20,9512 Rossrüti,9512,Rossrüti


In [192]:
df_laerm.tail(5)

,GPLZ,gemeinde,Geo Shape PLZ,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
10800,9608,Ganterschwil,"{""coordinates"": [[[9.0896852463, 47.3541665074...",MULTILINESTRING Z ((1250211.145300001 2724401....,Laermemission.3657,3657,GS,77,63,4500.0,...,NaN,NaN,NaN,1250201.827 2724400.940000001 0,"(47.39089007950691, 9.086388896549742)","Letzistrasse 209, 9608 Ganterschwil, Switzerland",Letzistrasse 209,9608 Ganterschwil,9608,Ganterschwil
10801,9608,Ganterschwil,"{""coordinates"": [[[9.0896852463, 47.3541665074...",MULTILINESTRING Z ((1250156.846000001 2724395....,Laermemission.3967,3967,GS,78,64,4500.0,...,NaN,NaN,NaN,1250068.208299998 2724384.701699998 0,"(47.38969161771949, 9.08613671183817)","Letzi 519, 9608 Ganterschwil, Switzerland",Letzi 519,9608 Ganterschwil,9608,Ganterschwil
10802,9608,Ganterschwil,"{""coordinates"": [[[9.0896852463, 47.3541665074...",MULTILINESTRING Z ((1250032.225699998 2724380....,Laermemission.3968,3968,GS,77,64,4500.0,...,NaN,NaN,NaN,1250008.25 2724377.4102 0,"(47.38915383632983, 9.086023488082006)","Letzistrasse, 9608 Ganterschwil, Switzerland",Letzistrasse,9608 Ganterschwil,9608,Ganterschwil
10803,9608,Ganterschwil,"{""coordinates"": [[[9.0896852463, 47.3541665074...",MULTILINESTRING Z ((1250390.2837 2724258.11890...,Laermemission.3971,3971,GS,77,64,4500.0,...,NaN,NaN,NaN,1250383.155299999 2724273.627500001 0,"(47.39254466024865, 9.084753452442555)","Letzistrasse, 9608 Ganterschwil, Switzerland",Letzistrasse,9608 Ganterschwil,9608,Ganterschwil
10804,9608,Ganterschwil,"{""coordinates"": [[[9.0896852463, 47.3541665074...",MULTILINESTRING Z ((1250366.707899999 2724303....,Laermemission.3972,3972,GS,77,64,4500.0,...,NaN,NaN,NaN,1250363.265999999 2724308.385000002 0,"(47.39235925135821, 9.085208186135873)","Letzistrasse, 9608 Ganterschwil, Switzerland",Letzistrasse,9608 Ganterschwil,9608,Ganterschwil


In [193]:
df_laerm.dtypes

GPLZ                                        int64
gemeinde                                   object
Geo Shape PLZ                              object
geometry                                   object
gml_id                                     object
OBJECTID                                    int64
Strasseneigentümer                         object
Emissionswert Tag                           int64
Emissionswert Nacht                         int64
Tagesverkehr Jahresmittel                 float64
Anzahl Fz Tag pro h                       float64
Anzahl Fz Nacht pro h                     float64
Prozentanteil lauter Fz Tag               float64
Prozentanteil lauter Fz Nacht             float64
Steigung                                  float64
Belagskorrektur                             int64
Tempo                                     float64
Faktor_Tag                                float64
Faktor_Nacht                              float64
Tunnel                                     object


In [194]:
df_laerm["Emissionswert Tag"] = df_laerm['Emissionswert Tag'].apply(lambda x: np.int64(x)) 
df_laerm["Emissionswert Nacht"] = df_laerm['Emissionswert Nacht'].apply(lambda x: np.int64(x)) 

In [195]:
df_laerm.head(2)

,GPLZ,gemeinde,Geo Shape PLZ,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
0,9000,St.Gallen,NaN,MULTILINESTRING Z ((1253168.0867 2745272.4672 ...,Laermemission.4156,4156,KS,79,71,14700.0,...,NaN,NaN,NaN,1253171.2366 2745271.824200001 0,"(47.413316295301385, 9.363697886370224)","Teufener Str. 123, 9000 St.Gallen, Switzerland",Teufener Str. 123,9000 St.Gallen,9000,St.Gallen
2,9000,St.Gallen,NaN,MULTILINESTRING Z ((1253172.473200001 2744061....,Laermemission.8233,8233,GS,74,59,3800.0,...,NaN,NaN,NaN,1253146.8541 2744072.648499999 0,"(47.4133606190565, 9.347805434262195)","Fürstenlandstrasse 53, 9000 St.Gallen, Switzer...",Fürstenlandstrasse 53,9000 St.Gallen,9000,St.Gallen


## 2. Auswertungen

- Wo ist es am lautesten (Tag und Nacht)?
- evlt: Wo ist es am stillsten (Tag und Nacht)?
- Kategorien nach Grenzwerten setzen
- Überraschendes?
- Lärm nach PLZ auswerten

### 2.1 Wo ist es generell am lautesten?

Aus Neugierde, evtl später interessant für andere Redaktionen. Aber Achtung, diese Daten enthalten noch Doppelte (mehr dazu unten).

In [196]:
#Wo ist es am lautesten, am Tag und in der Nacht?
df_laerm[['Emissionswert Tag', 'Emissionswert Nacht']].max()

Emissionswert Tag      83
Emissionswert Nacht    76
dtype: int64

In [197]:
#Am lautesten ist es am Tag mit 83 Dezibeln an folgenden Stellen
df_laerm[df_laerm['Emissionswert Tag'] == 83].head(10)

,GPLZ,gemeinde,Geo Shape PLZ,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
3854,9327,Tübach,"{""coordinates"": [[[9.4427382253, 47.4837962195...",MULTILINESTRING Z ((1260402.019000001 2752102....,Laermemission.2184,2184,KS,83,74,21500.0,...,NaN,NaN,NaN,1260398.149999999 2752114.484999999 0,"(47.47674828472033, 9.456797704514413)","St.Gallerstrasse 2, 9327 Tübach, Switzerland",St.Gallerstrasse 2,9327 Tübach,9327,Tübach
8965,9533,Dietschwil,"{""coordinates"": [[[8.9942428318, 47.4223354566...",MULTILINESTRING Z ((1255261.250500001 2722133....,Laermemission.5975,5975,KS,83,75,18900.0,...,NaN,NaN,NaN,1255241.538899999 2722145.515500002 0,"(47.43663047627584, 9.057900669311142)","Wilerstrasse, 9533 Kirchberg, Switzerland",Wilerstrasse,9533 Kirchberg,9533,Kirchberg
9013,9533,Dietschwil,"{""coordinates"": [[[8.9942428318, 47.4223354566...",MULTILINESTRING Z ((1255042.306600001 2722273....,Laermemission.9881,9881,KS,83,76,18900.0,...,NaN,NaN,NaN,1255034.622000001 2722278.120000001 0,"(47.434745218880266, 9.059601428938544)","Wilerstrasse 959, 9533 Kirchberg, Switzerland",Wilerstrasse 959,9533 Kirchberg,9533,Kirchberg
9089,9533,Kirchberg SG,"{""coordinates"": [[[9.0266439095, 47.397289575]...",MULTILINESTRING Z ((1255261.250500001 2722133....,Laermemission.5975,5975,KS,83,75,18900.0,...,NaN,NaN,NaN,1255241.538899999 2722145.515500002 0,"(47.43663047627584, 9.057900669311142)","Wilerstrasse, 9533 Kirchberg, Switzerland",Wilerstrasse,9533 Kirchberg,9533,Kirchberg
9137,9533,Kirchberg SG,"{""coordinates"": [[[9.0266439095, 47.397289575]...",MULTILINESTRING Z ((1255042.306600001 2722273....,Laermemission.9881,9881,KS,83,76,18900.0,...,NaN,NaN,NaN,1255034.622000001 2722278.120000001 0,"(47.434745218880266, 9.059601428938544)","Wilerstrasse 959, 9533 Kirchberg, Switzerland",Wilerstrasse 959,9533 Kirchberg,9533,Kirchberg
10286,9403,Goldach,"{""coordinates"": [[[9.4443068217, 47.4647129922...",MULTILINESTRING Z ((1260389.530400001 2752235....,Laermemission.9188,9188,KS,83,74,21500.0,...,NaN,NaN,NaN,1260392.090999998 2752244.874000002 0,"(47.47666362810701, 9.458524800313297)","St.Gallerstrasse 117B, 9403 Goldach, Switzerland",St.Gallerstrasse 117B,9403 Goldach,9403,Goldach


In [198]:
#Am lautesten ist es in der Nacht mit 76 Dezibeln an folgenden Stellen:
df_laerm[df_laerm['Emissionswert Nacht'] == 76].head(10)

,GPLZ,gemeinde,Geo Shape PLZ,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
9013,9533,Dietschwil,"{""coordinates"": [[[8.9942428318, 47.4223354566...",MULTILINESTRING Z ((1255042.306600001 2722273....,Laermemission.9881,9881,KS,83,76,18900.0,...,NaN,NaN,NaN,1255034.622000001 2722278.120000001 0,"(47.434745218880266, 9.059601428938544)","Wilerstrasse 959, 9533 Kirchberg, Switzerland",Wilerstrasse 959,9533 Kirchberg,9533,Kirchberg
9137,9533,Kirchberg SG,"{""coordinates"": [[[9.0266439095, 47.397289575]...",MULTILINESTRING Z ((1255042.306600001 2722273....,Laermemission.9881,9881,KS,83,76,18900.0,...,NaN,NaN,NaN,1255034.622000001 2722278.120000001 0,"(47.434745218880266, 9.059601428938544)","Wilerstrasse 959, 9533 Kirchberg, Switzerland",Wilerstrasse 959,9533 Kirchberg,9533,Kirchberg


### 2.2 Neuer Datensatz, der nur Stadtsanktgaller Daten enthält ...

In [199]:
df_sg = df_laerm[df_laerm['gemeinde'] == 'St.Gallen'].copy()

In [200]:
df_sg['strasse'].value_counts()

Teufener Str. 1              6
Geltenwilenstrasse 2         6
Tellstrasse 16               6
Winkelriedstrasse 27         6
Parkstrasse 2                6
                            ..
St.Georgen-Strasse 98        1
St.Georgen-Strasse 113-97    1
St.Georgen-Strasse 108       1
St.Georgen-Strasse 112       1
Vogelherdweg 6               1
Name: strasse, Length: 790, dtype: int64

### 2.3 Für denselben Standort sind mehrere Daten vorhanden:

Einige sind einfach wirklich doppelt, bei diesen entsprechen sich auch die Kooridnaten in der Spalte "geometry". Bei den anderen handelt es sich um eine Überschneidung.

Dabei setzt das Stück A der Zürcher Strasse an das Stück B an. An genau diesen Punkten sind beim Datensatz zwei Emissionswerte hinterlegt.

1. Die Doppelte, die tatsächlich doppelt sind, löschen
2. Die falschen Doppelten richtig zuordnen.

In [201]:
df_sg.sort_values('strasse', ascending = False).head(50)

,GPLZ,gemeinde,Geo Shape PLZ,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
739,9000,St.Gallen,"{""coordinates"": [[[9.4060740511, 47.4311079592...",MULTILINESTRING Z ((1253440.049400002 2743924....,Laermemission.9075,9075,KS,79,70,16300.0,...,NaN,NaN,NaN,1253442.601100001 2743946.8807 0,"(47.416047334782185, 9.346234762201917)","Zürcherstrasse 99, 9000 St.Gallen, Switzerland",Zürcherstrasse 99,9000 St.Gallen,9000,St.Gallen
219,9000,St.Gallen,NaN,MULTILINESTRING Z ((1253440.049400002 2743924....,Laermemission.9075,9075,KS,79,70,16300.0,...,NaN,NaN,NaN,1253442.601100001 2743946.8807 0,"(47.416047334782185, 9.346234762201917)","Zürcherstrasse 99, 9000 St.Gallen, Switzerland",Zürcherstrasse 99,9000 St.Gallen,9000,St.Gallen
737,9000,St.Gallen,"{""coordinates"": [[[9.4060740511, 47.4311079592...",MULTILINESTRING Z ((1253464.386700001 2744035....,Laermemission.9073,9073,KS,79,70,16300.0,...,NaN,NaN,NaN,1253496.896299999 2744092.655099999 0)),"(47.416503632281156, 9.348183337732957)","Zürcherstrasse 91, 9000 St.Gallen, Switzerland",Zürcherstrasse 91,9000 St.Gallen,9000,St.Gallen
217,9000,St.Gallen,NaN,MULTILINESTRING Z ((1253464.386700001 2744035....,Laermemission.9073,9073,KS,79,70,16300.0,...,NaN,NaN,NaN,1253496.896299999 2744092.655099999 0)),"(47.416503632281156, 9.348183337732957)","Zürcherstrasse 91, 9000 St.Gallen, Switzerland",Zürcherstrasse 91,9000 St.Gallen,9000,St.Gallen
9431,9000,St.Gallen,"{""coordinates"": [[[9.2855886438, 47.4022789703...",MULTILINESTRING Z ((1252607.5022 2739428.62029...,Laermemission.6469,6469,KS,79,72,17900.0,...,NaN,NaN,NaN,1252589.567000002 2739471.0689 0,"(47.40934076544206, 9.28667658755805)","Zürcherstrasse 601, 9015 St.Gallen, Switzerland",Zürcherstrasse 601,9015 St.Gallen,9015,St.Gallen
9483,9000,St.Gallen,"{""coordinates"": [[[9.2855886438, 47.4022789703...",MULTILINESTRING Z ((1252410.1404 2740021.71550...,Laermemission.9238,9238,KS,79,71,16000.0,...,NaN,NaN,NaN,1252405.657499999 2740109.212099999 0,"(47.40755161113499, 9.295071445503208)","Zürcherstrasse 511, 9015 St.Gallen, Switzerland",Zürcherstrasse 511,9015 St.Gallen,9015,St.Gallen
9484,9000,St.Gallen,"{""coordinates"": [[[9.2855886438, 47.4022789703...",MULTILINESTRING Z ((1252413.513799999 2740252....,Laermemission.9239,9239,KS,79,71,16000.0,...,NaN,NaN,NaN,1252423.405000001 2740333.427900001 0,"(47.40766343307403, 9.298046762526992)","Zürcherstrasse 499, 9015 St.Gallen, Switzerland",Zürcherstrasse 499,9015 St.Gallen,9015,St.Gallen
9447,9000,St.Gallen,"{""coordinates"": [[[9.2855886438, 47.4022789703...",MULTILINESTRING Z ((1252397.8814 2741171.62979...,Laermemission.9097,9097,KS,80,71,21000.0,...,NaN,NaN,NaN,1252397.0616 2741212.619899999 0,"(47.407238562843204, 9.309683343601424)","Zürcherstrasse 431, 9015 St.Gallen, Switzerland",Zürcherstrasse 431,9015 St.Gallen,9015,St.Gallen
1000,9000,St.Gallen,"{""coordinates"": [[[9.4060740511, 47.4311079592...",MULTILINESTRING Z ((1253773.9989 2744481.9637 ...,Laermemission.10446,10446,GS,68,53,2700.0,...,NaN,NaN,NaN,1253752.488400001 2744500.837699998 0,"(47.41871229031177, 9.353673215415332)","Zürcherstrasse 41, 9000 St.Gallen, Switzerland",Zürcherstrasse 41,9000 St.Gallen,9000,St.Gallen
480,9000,St.Gallen,NaN,MULTILINESTRING Z ((1253773.9989 2744481.9637 ...,Laermemission.10446,10446,GS,68,53,2700.0,...,NaN,NaN,NaN,1253752.488400001 2744500.837699998 0,"(47.41871229031177, 9.353673215415332)","Zürcherstrasse 41, 9000 St.Gallen, Switzerland",Zürcherstrasse 41,9000 St.Gallen,9000,St.Gallen


In [202]:
#Spalte verhindert, dass drop_duplicates funktoniert
df_sg.drop('Geo Shape PLZ', inplace = True, axis = 1)

In [203]:
df_sg = df_sg.drop_duplicates().copy()

In [204]:
#Die "falschen Doppelten" sind noch da)
df_sg[df_sg['strasse'] == 'Zürcherstrasse 152']

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
9342,9000,St.Gallen,MULTILINESTRING Z ((1253546.238400001 2743637....,Laermemission.10489,10489,GS,71,56,2000.0,115.0,...,NaN,NaN,NaN,1253540.826900002 2743591.426800001 0,"(47.41700816870393, 9.341557647580672)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen
9343,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,NaN,NaN,NaN,1253504.6501 2743435.577500001 0,"(47.41671686641098, 9.339481464188031)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen


In [205]:
#101 Stellen sind doppelt -> zweite Kooridnaten dazunehmen
len(df_sg['strasse'])-len(df_sg['strasse'].drop_duplicates())

100

In [206]:
df_sg.to_csv('daten/df_sg-without-duplicates.csv', index = False)

#### Falsche Doppelte berichtigen

In [16]:
df_sg = pd.read_csv('daten/df_sg-without-duplicates.csv')

Doppelte in eigenes df packen. Aber nicht alle, sondern nur jene, die an erster Stelle vorkommen. Diese werden bereinigt. Jene, die an zweiter Stelle vorkommen, behalte ich.

In [17]:
df_duplicates = df_sg[df_sg.duplicated(['strasse', 'addresses'], keep = 'first')].sort_values('strasse')

In [18]:
df_duplicates[df_duplicates['strasse'] == 'Zürcherstrasse 152']

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
699,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,NaN,NaN,NaN,1253504.6501 2743435.577500001 0,"(47.41671686641098, 9.339481464188031)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen


In [19]:
df_duplicates.sort_values('strasse', ascending = False).head(30)

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
658,9000,St.Gallen,MULTILINESTRING Z ((1252470.9142 2742496.06569...,Laermemission.9355,9355,GS,73,59,4500.0,262.0,...,NaN,NaN,NaN,1252489.0634 2742535.659299999 0)),"(47.40778067937879, 9.327236118441531)","Zürcherstrasse 297, 9014 St.Gallen, Switzerland",Zürcherstrasse 297,9014 St.Gallen,9014,St.Gallen
699,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,NaN,NaN,NaN,1253504.6501 2743435.577500001 0,"(47.41671686641098, 9.339481464188031)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen
328,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,Laermemission.9574,9574,GS,72,57,2100.0,119.0,...,NaN,NaN,NaN,1253959.293200001 2743819.7588 0,"(47.4207209420824, 9.344717126984364)","Zwyssigstrasse 28, 9000 St.Gallen, Switzerland",Zwyssigstrasse 28,9000 St.Gallen,9000,St.Gallen
152,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....,Laermemission.8812,8812,GS,71,57,2800.0,161.0,...,NaN,NaN,NaN,1254575.526099999 2745700.4364 0,"(47.42584808060903, 9.369833381835788)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen
240,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,Laermemission.9242,9242,GS,61,49,1000.0,54.0,...,NaN,NaN,NaN,1254551.704100002 2745676.312100001 0,"(47.425639228695424, 9.369505981703966)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen
202,9000,St.Gallen,MULTILINESTRING Z ((1254493.754000001 2746970....,Laermemission.9012,9012,KS,74,60,4400.0,253.0,...,NaN,62.9,56.5,1254496.261399999 2746980.327399999 0,"(47.42485077825742, 9.386764845789028)","Wildeggstrasse 46, 9000 St.Gallen, Switzerland",Wildeggstrasse 46,9000 St.Gallen,9000,St.Gallen
423,9000,St.Gallen,MULTILINESTRING Z ((1253709.467399999 2744538....,Laermemission.10445,10445,GS,69,54,2700.0,155.0,...,NaN,NaN,NaN,1253695.875300001 2744551.483600002 0,"(47.41819213618834, 9.354325832048065)","Turnerstrasse 35, 9000 St.Gallen, Switzerland",Turnerstrasse 35,9000 St.Gallen,9000,St.Gallen
490,9000,St.Gallen,MULTILINESTRING Z ((1253010.077399999 2745483....,Laermemission.8568,8568,GS,71,57,3700.0,213.0,...,NaN,NaN,NaN,1253017.977000002 2745504.753400002 0,"(47.411886826742354, 9.366733400947453)","Teufener Str. 143, 9012 St.Gallen, Switzerland",Teufener Str. 143,9012 St.Gallen,9012,St.Gallen
172,9000,St.Gallen,MULTILINESTRING Z ((1254086.5348 2745890.7355 ...,Laermemission.8868,8868,GS,71,57,3400.0,198.0,...,NaN,NaN,NaN,1254060.5011 2745897.2454 0,"(47.4211736968705, 9.372272724447075)","Teufener Str. 1, 9000 St.Gallen, Switzerland",Teufener Str. 1,9000 St.Gallen,9000,St.Gallen
319,9000,St.Gallen,MULTILINESTRING Z ((1254048.849399999 2745851....,Laermemission.9522,9522,GS,71,58,3800.0,218.0,...,NaN,NaN,NaN,1254080.090999998 2745872.233899999 0)),"(47.421355381086315, 9.371947766718792)","Teufener Str. 1, 9000 St.Gallen, Switzerland",Teufener Str. 1,9000 St.Gallen,9000,St.Gallen


In [20]:
#Alle ersten Vorkommnise der Doppelten aus df_sg löschen (später die bereinigten Daten wieder anfügen
df_sg_ohne = df_sg.drop_duplicates('strasse', keep = 'last').copy()

In [21]:
#Die bereits bereinigten Zeilen sind jetzt noch enthalten, von den "falschen Doppelten" aber nur noch je einer.
#df_sg_ohne[df_sg_ohne['strasse'] == 'Zürcherstrasse 155']
df_sg_ohne[df_sg_ohne['strasse'] == 'Zürcherstrasse 152']

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
699,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,NaN,NaN,NaN,1253504.6501 2743435.577500001 0,"(47.41671686641098, 9.339481464188031)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen


In [22]:
df_sg_ohne.to_csv('daten/df_sg_without duplicates_new.csv', index = False)

#### Koordinaten bei Geometry rausrechnen und neuer Punkt setzen

In [213]:
df_duplicates.reset_index(drop = True, inplace = True)

In [214]:
df_duplicates.head(1)

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
0,9000,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,Laermemission.8232,8232,GS,61,50,800.0,45.0,...,NaN,NaN,NaN,1255386.065900002 2748919.897700001 0,"(47.4324151893062, 9.412758669938288)","Achslenstrasse 12, 9016 St.Gallen, Switzerland",Achslenstrasse 12,9016 St.Gallen,9016,St.Gallen


In [215]:
# Split Koordinaten in geometry beim Komma und packt das Resultat in eine Liste
geopoint = []

for i in range(len(df_duplicates)):
    elem = df_duplicates['geometry'].str.split(",")[i]

  
    geopoint.append(elem)   

In [216]:
geopoint[1]

['MULTILINESTRING Z ((1255698.543400001 2747408.798900001 0',
 ' 1255687.495299999 2747382.293699998 0',
 ' 1255680.129900001 2747364.623500001 0))']

In [217]:
df_geopoint = pd.DataFrame(geopoint)

In [218]:
# Das df mit den Daten und jenes mit den Geopoints (Spalte 0!) auf dem Index mergen
df_new = pd.merge(df_duplicates,df_geopoint[[0]], left_index=True, right_index=True)

In [219]:
df_new.tail(4)

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname,0
95,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....,Laermemission.8812,8812,GS,71,57,2800.0,161.0,...,NaN,NaN,1254575.526099999 2745700.4364 0,"(47.42584808060903, 9.369833381835788)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....
96,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,Laermemission.9574,9574,GS,72,57,2100.0,119.0,...,NaN,NaN,1253959.293200001 2743819.7588 0,"(47.4207209420824, 9.344717126984364)","Zwyssigstrasse 28, 9000 St.Gallen, Switzerland",Zwyssigstrasse 28,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....
97,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,NaN,NaN,1253504.6501 2743435.577500001 0,"(47.41671686641098, 9.339481464188031)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....
98,9000,St.Gallen,MULTILINESTRING Z ((1252470.9142 2742496.06569...,Laermemission.9355,9355,GS,73,59,4500.0,262.0,...,NaN,NaN,1252489.0634 2742535.659299999 0)),"(47.40778067937879, 9.327236118441531)","Zürcherstrasse 297, 9014 St.Gallen, Switzerland",Zürcherstrasse 297,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1252470.9142 2742496.06569...


In [220]:
df_new.rename(columns={0: 'new_point'}, inplace=True)

#### Als nächstes die Koordinaten in einzelne Spalten packen

In [221]:
# Koordinaten in column new_point bei den Leerstellen splitten
coordinates = []

for i in range(len(df_new)):
    elem = df_new['new_point'].str.split(' ')[i]
    coordinates.append(elem) 

In [222]:
coordinates

[['MULTILINESTRING', 'Z', '((1255393.619600002', '2748935.004799999', '0'],
 ['MULTILINESTRING', 'Z', '((1255698.543400001', '2747408.798900001', '0'],
 ['MULTILINESTRING', 'Z', '((1255680.129900001', '2747364.623500001', '0'],
 ['MULTILINESTRING', 'Z', '((1251848.976799998', '2740792.215500001', '0'],
 ['MULTILINESTRING', 'Z', '((1252666.3255', '2741506.477000002', '0'],
 ['MULTILINESTRING', 'Z', '((1254080.090999998', '2745872.233899999', '0'],
 ['MULTILINESTRING', 'Z', '((1256369.017000001', '2747447.5636', '0'],
 ['MULTILINESTRING', 'Z', '((1254642.564800002', '2746324.873', '0'],
 ['MULTILINESTRING', 'Z', '((1255165.279199999', '2746264.309', '0'],
 ['MULTILINESTRING', 'Z', '((1254443.411200002', '2745035.209899999', '0'],
 ['MULTILINESTRING', 'Z', '((1254294.260299999', '2744877.982500002', '0'],
 ['MULTILINESTRING', 'Z', '((1255217.3728', '2746215.655200001', '0'],
 ['MULTILINESTRING', 'Z', '((1254843.615899999', '2745869.1932', '0'],
 ['MULTILINESTRING', 'Z', '((1253648.8737999

In [223]:
coordinates[10][2]

'((1254294.260299999'

In [224]:
coordinates[10][3]

'2744877.982500002'

In [225]:
# aus der Liste die Koordinaten auslesen 

long = []
lat = []

for i in range(len(coordinates)):
    elem_lo = coordinates[i][2]
    long.append(elem_lo)
    
    elem_lat = coordinates[i][3]
    lat.append(elem_lat)  

In [226]:
long

['((1255393.619600002',
 '((1255698.543400001',
 '((1255680.129900001',
 '((1251848.976799998',
 '((1252666.3255',
 '((1254080.090999998',
 '((1256369.017000001',
 '((1254642.564800002',
 '((1255165.279199999',
 '((1254443.411200002',
 '((1254294.260299999',
 '((1255217.3728',
 '((1254843.615899999',
 '((1253648.873799998',
 '((1255957.381200001',
 '((1255277.0748',
 '((1254886.975499999',
 '((1254910.572799999',
 '((1252866.281399999',
 '((1254129.781500001',
 '((1253922.202399999',
 '((1253942.8972',
 '((1254006.255100001',
 '((1253937.811500002',
 '((1253925.085999999',
 '((1252960.581999999',
 '((1252732.587000001',
 '((1252382.712699998',
 '((1252627.5473',
 '((1254803.721700002',
 '((1254186.6699',
 '((1252577.145100001',
 '((1252737.6578',
 '((1252896.675099999',
 '((1252896.675099999',
 '((1252458.581799999',
 '((1255921.6941',
 '((1255821.055399999',
 '((1255006.717300002',
 '((1251991.357700001',
 '((1256273.5671',
 '((1256303.1853',
 '((1252425.0933',
 '((1253062.104600001',

In [227]:
#mit Regex die Klammern in long entfernen
long_new = [re.sub(r'\D\D', '', i) for i in long]

In [228]:
long_new

['1255393.619600002',
 '1255698.543400001',
 '1255680.129900001',
 '1251848.976799998',
 '1252666.3255',
 '1254080.090999998',
 '1256369.017000001',
 '1254642.564800002',
 '1255165.279199999',
 '1254443.411200002',
 '1254294.260299999',
 '1255217.3728',
 '1254843.615899999',
 '1253648.873799998',
 '1255957.381200001',
 '1255277.0748',
 '1254886.975499999',
 '1254910.572799999',
 '1252866.281399999',
 '1254129.781500001',
 '1253922.202399999',
 '1253942.8972',
 '1254006.255100001',
 '1253937.811500002',
 '1253925.085999999',
 '1252960.581999999',
 '1252732.587000001',
 '1252382.712699998',
 '1252627.5473',
 '1254803.721700002',
 '1254186.6699',
 '1252577.145100001',
 '1252737.6578',
 '1252896.675099999',
 '1252896.675099999',
 '1252458.581799999',
 '1255921.6941',
 '1255821.055399999',
 '1255006.717300002',
 '1251991.357700001',
 '1256273.5671',
 '1256303.1853',
 '1252425.0933',
 '1253062.104600001',
 '1253023.199099999',
 '1256839.259199999',
 '1256224.867800001',
 '1256116.710000001',

In [229]:
d = {'longitude_new':long_new, 'latitude_new':lat}

In [230]:
df_coordinates = pd.DataFrame(d)

In [231]:
df_coordinates.head(2)

,longitude_new,latitude_new
0,1255393.619600002,2748935.004799999
1,1255698.543400001,2747408.798900001


In [232]:
df_final = pd.merge(df_new,df_coordinates, left_index=True, right_index=True)

In [233]:
df_final

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname,new_point,longitude_new,latitude_new
0,9000,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,Laermemission.8232,8232,GS,61,50,800.0,45.0,...,1255386.065900002 2748919.897700001 0,"(47.4324151893062, 9.412758669938288)","Achslenstrasse 12, 9016 St.Gallen, Switzerland",Achslenstrasse 12,9016 St.Gallen,9016,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,1255393.619600002,2748935.004799999
1,9000,St.Gallen,MULTILINESTRING Z ((1255698.543400001 2747408....,Laermemission.9568,9568,GS,73,61,5300.0,300.0,...,1255687.495299999 2747382.293699998 0,"(47.435471617721426, 9.39248365328597)","Bachstrasse, 9008 St.Gallen, Switzerland",Bachstrasse,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255698.543400001 2747408....,1255698.543400001,2747408.798900001
2,9000,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,Laermemission.9569,9569,GS,73,61,5300.0,300.0,...,1255658.0337 2747311.613000002 0)),"(47.4352225483372, 9.391537300282904)","Bachstrasse 29, 9008 St.Gallen, Switzerland",Bachstrasse 29,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,1255680.129900001,2747364.623500001
3,9000,St.Gallen,MULTILINESTRING Z ((1251848.976799998 2740792....,Laermemission.9484,9484,GS,76,67,7900.0,454.0,...,1251844.7194 2740797.898600001 0)),"(47.40236094076094, 9.304016398564753)","Biderstrasse 15, 9015 St.Gallen, Switzerland",Biderstrasse 15,9015 St.Gallen,9015,St.Gallen,MULTILINESTRING Z ((1251848.976799998 2740792....,1251848.976799998,2740792.215500001
4,9000,St.Gallen,MULTILINESTRING Z ((1252666.3255 2741506.47700...,Laermemission.9806,9806,GS,68,53,1800.0,105.0,...,1252668.480500001 2741515.621800002 0,"(47.409613983850754, 9.313782516058682)","Bionstrasse 4, 9015 St.Gallen, Switzerland",Bionstrasse 4,9015 St.Gallen,9015,St.Gallen,MULTILINESTRING Z ((1252666.3255 2741506.47700...,1252666.3255,2741506.477000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,Laermemission.9242,9242,GS,61,49,1000.0,54.0,...,1254551.704100002 2745676.312100001 0,"(47.425639228695424, 9.369505981703966)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,1254556.100299999,2745691.578600001
95,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....,Laermemission.8812,8812,GS,71,57,2800.0,161.0,...,1254575.526099999 2745700.4364 0,"(47.42584808060903, 9.369833381835788)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....,1254569.050799999,2745697.483800001
96,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,Laermemission.9574,9574,GS,72,57,2100.0,119.0,...,1253959.293200001 2743819.7588 0,"(47.4207209420824, 9.344717126984364)","Zwyssigstrasse 28, 9000 St.Gallen, Switzerland",Zwyssigstrasse 28,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,1253969.268300001,2743817.853399999
97,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,1253504.6501 2743435.577500001 0,"(47.41671686641098, 9.339481464188031)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,1253511.328899998,2743462.295000002


In [234]:
df_final.to_csv('daten/df_sg-duplicates_with_coordinates.csv', index = False)

In [235]:
df_final = pd.read_csv('daten/df_sg-duplicates_with_coordinates.csv')

### Koordinaten umwandeln

In [236]:
import pandas as pd
from pyproj import Transformer #Für Umrechnen der Koordinaten

import geopy
from geopy.geocoders import GoogleV3 #der Google-API (Addressabfrage)

from geopy.extra.rate_limiter import RateLimiter #Verzögern der Abfrage

In [237]:
transformer = Transformer.from_crs("epsg:2056", "epsg:4326")

In [238]:
gps = []

for i in range(len(df_final)): #liest Spalten latitude und longitude aus,
    #transformiert diese in EPSG4326
    r = transformer.transform(df_final['latitude_new'][i], df_final['longitude_new'][i])
    
    gps.append(r) #packt die Ergebnisse in die List gps

In [239]:
gps[3][1]

9.303942474169684

In [240]:
df_final['gps_new'] = gps

In [241]:
df_final[df_final['strasse'] == "Bachstrasse 29"]

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,gps,addresses,strasse,addressteil,plz,dorfname,new_point,longitude_new,latitude_new,gps_new
2,9000,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,Laermemission.9569,9569,GS,73,61,5300.0,300.0,...,"(47.4352225483372, 9.391537300282904)","Bachstrasse 29, 9008 St.Gallen, Switzerland",Bachstrasse 29,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,1.255680e+06,2.747365e+06,"(47.43540935110947, 9.392247063873524)"


In [242]:
df_final.head(10)

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,gps,addresses,strasse,addressteil,plz,dorfname,new_point,longitude_new,latitude_new,gps_new
0,9000,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,Laermemission.8232,8232,GS,61,50,800.0,45.0,...,"(47.4324151893062, 9.412758669938288)","Achslenstrasse 12, 9016 St.Gallen, Switzerland",Achslenstrasse 12,9016 St.Gallen,9016,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,1.255394e+06,2.748935e+06,"(47.43247968717648, 9.412961369825302)"
1,9000,St.Gallen,MULTILINESTRING Z ((1255698.543400001 2747408....,Laermemission.9568,9568,GS,73,61,5300.0,300.0,...,"(47.435471617721426, 9.39248365328597)","Bachstrasse, 9008 St.Gallen, Switzerland",Bachstrasse,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255698.543400001 2747408....,1.255699e+06,2.747409e+06,"(47.4355650167513, 9.39283853711761)"
2,9000,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,Laermemission.9569,9569,GS,73,61,5300.0,300.0,...,"(47.4352225483372, 9.391537300282904)","Bachstrasse 29, 9008 St.Gallen, Switzerland",Bachstrasse 29,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,1.255680e+06,2.747365e+06,"(47.43540935110947, 9.392247063873524)"
3,9000,St.Gallen,MULTILINESTRING Z ((1251848.976799998 2740792....,Laermemission.9484,9484,GS,76,67,7900.0,454.0,...,"(47.40236094076094, 9.304016398564753)","Biderstrasse 15, 9015 St.Gallen, Switzerland",Biderstrasse 15,9015 St.Gallen,9015,St.Gallen,MULTILINESTRING Z ((1251848.976799998 2740792....,1.251849e+06,2.740792e+06,"(47.402400437014414, 9.303942474169684)"
4,9000,St.Gallen,MULTILINESTRING Z ((1252666.3255 2741506.47700...,Laermemission.9806,9806,GS,68,53,1800.0,105.0,...,"(47.409613983850754, 9.313782516058682)","Bionstrasse 4, 9015 St.Gallen, Switzerland",Bionstrasse 4,9015 St.Gallen,9015,St.Gallen,MULTILINESTRING Z ((1252666.3255 2741506.47700...,1.252666e+06,2.741506e+06,"(47.4095965728011, 9.313660706439233)"
5,9000,St.Gallen,MULTILINESTRING Z ((1254080.090999998 2745872....,Laermemission.9520,9520,GS,71,58,3800.0,218.0,...,"(47.42156469028934, 9.372137570913823)","Bleichestrasse 9, 9000 St.Gallen, Switzerland",Bleichestrasse 9,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254080.090999998 2745872....,1.254080e+06,2.745872e+06,"(47.421355381086315, 9.371947766718792)"
6,9000,St.Gallen,MULTILINESTRING Z ((1256369.017000001 2747447....,Laermemission.9478,9478,KS,78,70,14800.0,828.0,...,"(47.44216115410511, 9.394262377504909)","Bruggwaldstrasse 1A, 9008 St.Gallen, Switzerland",Bruggwaldstrasse 1A,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1256369.017000001 2747447....,1.256369e+06,2.747448e+06,"(47.44158473860958, 9.393573696917437)"
7,9000,St.Gallen,MULTILINESTRING Z ((1254642.564800002 2746324....,Laermemission.8729,8729,KS,75,68,8200.0,457.0,...,"(47.42629730015778, 9.378372446384605)","Brühltor-Passage 1, 9000 St.Gallen, Switzerland",Brühltor-Passage 1,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254642.564800002 2746324....,1.254643e+06,2.746325e+06,"(47.42631230469604, 9.378128663373413)"
8,9000,St.Gallen,MULTILINESTRING Z ((1255165.279199999 2746264....,Laermemission.10023,10023,GS,70,54,2600.0,152.0,...,"(47.43090993331156, 9.37738831204976)","Dierauerstrasse, 9000 St.Gallen, Switzerland",Dierauerstrasse,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1255165.279199999 2746264....,1.255165e+06,2.746264e+06,"(47.431025672489206, 9.377497437337881)"
9,9000,St.Gallen,MULTILINESTRING Z ((1254443.411200002 2745035....,Laermemission.8572,8572,GS,56,46,600.0,32.0,...,"(47.42491611766419, 9.361130907190459)","Dietlistrasse 44, 9000 St.Gallen, Switzerland",Dietlistrasse 44,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254443.411200002 2745035....,1.254443e+06,2.745035e+06,"(47.424807125513176, 9.360976678253502)"


#### Aus den Koordinaten via Google-API Adressen ziehen

In [98]:
# Google-API geocoding als gelocater setzen (API-Key mitgeben)
geolocator = GoogleV3(api_key = 'XXX')

In [243]:
df_final.shape

(99, 35)

In [244]:
#RateLlimiter auf zwei Sekunden setzen
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)


addresses = []

for i in range(len(df_final)):
    #speichert aufgrund der Koordinaten das Adress-File
    location = geolocator.reverse(df_final['gps_new'].iloc[i])
    
    #holt aus dem Adressen-File die raw-Daten
    value = location.address
    
    #packt die raw-Daten in eine Liste
    addresses.append(value)

In [245]:
addresses

['Achslenstrasse 12, 9016 St. Gallen, Switzerland',
 'Bachstrasse, 9008 St. Gallen, Switzerland',
 'Bachstrasse 29, 9008 St. Gallen, Switzerland',
 'Biderstrasse 15, 9015 St. Gallen, Switzerland',
 'Piccardstrasse 1, 9015 St. Gallen, Switzerland',
 'Teufener Str. 1, 9000 St. Gallen, Switzerland',
 'Langgasse 108, 9008 St. Gallen, Switzerland',
 'Brühltor-Passage 1, 9000 St. Gallen, Switzerland',
 'Müller-Friedberg-Strasse 33, 9000 St. Gallen, Switzerland',
 'Dietlistrasse 44, 9000 St. Gallen, Switzerland',
 'Dietlistrasse 6, 9000 St. Gallen, Switzerland',
 'Dufourstrasse 45, 9000 St. Gallen, Switzerland',
 'Tigerberg, 9000 St. Gallen, Switzerland',
 'Dürrenmattstrasse 30, 9000 St. Gallen, Switzerland',
 'Favrestrasse 7, 9016 St. Gallen, Switzerland',
 'Flurhofstrasse 145, 9000 St. Gallen, Switzerland',
 'Flurhofstrasse 49, 9000 St. Gallen, Switzerland',
 'Flurhofstrasse 53, 9000 St. Gallen, Switzerland',
 'Fürstenlandstrasse 129, 9014 St. Gallen, Switzerland',
 'Wassergasse 3, 9000 St.

In [246]:
#mit Regex den Leerschlag zwischen St. und Strassennamen oder Gemeindenamen entfernen
address_new = [re.sub(r'St. ', 'St.', i) for i in addresses]

In [247]:
df_final['addresses_new'] = address_new

In [248]:
df_final

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,addresses,strasse,addressteil,plz,dorfname,new_point,longitude_new,latitude_new,gps_new,addresses_new
0,9000,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,Laermemission.8232,8232,GS,61,50,800.0,45.0,...,"Achslenstrasse 12, 9016 St.Gallen, Switzerland",Achslenstrasse 12,9016 St.Gallen,9016,St.Gallen,MULTILINESTRING Z ((1255393.619600002 2748935....,1.255394e+06,2.748935e+06,"(47.43247968717648, 9.412961369825302)","Achslenstrasse 12, 9016 St.Gallen, Switzerland"
1,9000,St.Gallen,MULTILINESTRING Z ((1255698.543400001 2747408....,Laermemission.9568,9568,GS,73,61,5300.0,300.0,...,"Bachstrasse, 9008 St.Gallen, Switzerland",Bachstrasse,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255698.543400001 2747408....,1.255699e+06,2.747409e+06,"(47.4355650167513, 9.39283853711761)","Bachstrasse, 9008 St.Gallen, Switzerland"
2,9000,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,Laermemission.9569,9569,GS,73,61,5300.0,300.0,...,"Bachstrasse 29, 9008 St.Gallen, Switzerland",Bachstrasse 29,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255680.129900001 2747364....,1.255680e+06,2.747365e+06,"(47.43540935110947, 9.392247063873524)","Bachstrasse 29, 9008 St.Gallen, Switzerland"
3,9000,St.Gallen,MULTILINESTRING Z ((1251848.976799998 2740792....,Laermemission.9484,9484,GS,76,67,7900.0,454.0,...,"Biderstrasse 15, 9015 St.Gallen, Switzerland",Biderstrasse 15,9015 St.Gallen,9015,St.Gallen,MULTILINESTRING Z ((1251848.976799998 2740792....,1.251849e+06,2.740792e+06,"(47.402400437014414, 9.303942474169684)","Biderstrasse 15, 9015 St.Gallen, Switzerland"
4,9000,St.Gallen,MULTILINESTRING Z ((1252666.3255 2741506.47700...,Laermemission.9806,9806,GS,68,53,1800.0,105.0,...,"Bionstrasse 4, 9015 St.Gallen, Switzerland",Bionstrasse 4,9015 St.Gallen,9015,St.Gallen,MULTILINESTRING Z ((1252666.3255 2741506.47700...,1.252666e+06,2.741506e+06,"(47.4095965728011, 9.313660706439233)","Piccardstrasse 1, 9015 St.Gallen, Switzerland"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,Laermemission.9242,9242,GS,61,49,1000.0,54.0,...,"Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,1.254556e+06,2.745692e+06,"(47.42567537701819, 9.369709685247644)","Zwinglistrasse 21, 9000 St.Gallen, Switzerland"
95,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....,Laermemission.8812,8812,GS,71,57,2800.0,161.0,...,"Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1254569.050799999 2745697....,1.254569e+06,2.745697e+06,"(47.425790512461504, 9.36979214953966)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland"
96,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,Laermemission.9574,9574,GS,72,57,2100.0,119.0,...,"Zwyssigstrasse 28, 9000 St.Gallen, Switzerland",Zwyssigstrasse 28,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,1.253969e+06,2.743818e+06,"(47.420811049138294, 9.344695095627596)","Zwyssigstrasse 28, 9000 St.Gallen, Switzerland"
97,9000,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,Laermemission.10490,10490,GS,69,54,2000.0,115.0,...,"Zürcherstrasse 152, 9014 St.Gallen, Switzerland",Zürcherstrasse 152,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1253511.328899998 2743462....,1.253511e+06,2.743462e+06,"(47.41677109599754, 9.33983753924148)","Zürcherstrasse 152, 9014 St.Gallen, Switzerland"


In [249]:
# Adressen beim Komma splitten, um Strassen und Gemeindenamen seperat zu erhalten
address_list = []

for i in range(len(df_final)):
    
    elem = df_final['addresses_new'].iloc[i].split(',')
    
    address_list.append(elem)

In [250]:
#mit for-loop Strassen und Gemeinden aus der Liste auslesen und in seperate Listen packen
strasse = []
gemeinde = []

for i in range(len(address_list)):
    elem_str = address_list[i][0]
    strasse.append(elem_str)
    
    elem_gem = address_list[i][1]
    gemeinde.append(elem_gem)

In [251]:
gemeinde

[' 9016 St.Gallen',
 ' 9008 St.Gallen',
 ' 9008 St.Gallen',
 ' 9015 St.Gallen',
 ' 9015 St.Gallen',
 ' 9000 St.Gallen',
 ' 9008 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9016 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9014 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9015 St.Gallen',
 ' 9015 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9008 St.Gallen',
 ' 9008 St.Gallen',
 ' 9000 St.Gallen',
 ' 9015 St.Gallen',
 ' 9016 St.Gallen',
 ' 9008 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9014 St.Gallen',
 ' 9016 St.Gallen',
 ' 9008 St.Gallen',
 ' 9008 St.Gallen',
 ' 9000 St.Gallen',
 ' 9000 St.Gallen',


In [252]:
#Strasse und Gemeinde dem df hinzufügen
df_final['strasse_new'] = strasse
df_final['gemeinde_new'] = gemeinde

In [253]:
#In den beiden Spalten 0 durch NaN ersetzen
cols = ["Emissionswert Tag","Emissionswert Nacht"]
df_final[cols] = df_final[cols].replace({'0':np.nan, 0:np.nan})

#rows mit NaN in diesen beiden Spalten entfernen
df_final.dropna(subset=["Emissionswert Tag","Emissionswert Nacht"], inplace = True)

In [255]:
#Im neuen df sind jetzt noch 6 doppelt vorhanden. Das lässt sich auch von Hand bereinigen.
len(df_final['strasse_new'])-len(df_final['strasse_new'].drop_duplicates())

6

In [254]:
df_final.to_csv('daten/df_duplicates_with addresses.csv', index = False)

In [256]:
df_final[df_final["strasse_new"].duplicated()]

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,addressteil,plz,dorfname,new_point,longitude_new,latitude_new,gps_new,addresses_new,strasse_new,gemeinde_new
21,9000,St.Gallen,MULTILINESTRING Z ((1253942.8972 2745441.3607 ...,Laermemission.9494,9494,KS,80,72,25000.0,1421.0,...,9000 St.Gallen,9000,St.Gallen,MULTILINESTRING Z ((1253942.8972 2745441.3607 0,1.253943e+06,2.745441e+06,"(47.42021717444648, 9.366194810250766)","Geltenwilenstrasse 2, 9000 St.Gallen, Switzerland",Geltenwilenstrasse 2,9000 St.Gallen
34,9000,St.Gallen,MULTILINESTRING Z ((1252896.675099999 2744007....,Laermemission.10330,10330,GS,63,52,900.0,51.0,...,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1252896.675099999 2744007....,1.252897e+06,2.744008e+06,"(47.411125291215164, 9.3468681943638)","Hechtackerstrasse 39, 9014 St.Gallen, Switzerland",Hechtackerstrasse 39,9014 St.Gallen
44,9000,St.Gallen,MULTILINESTRING Z ((1253023.199099999 2743585....,Laermemission.9584,9584,GS,71,56,2600.0,152.0,...,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1253023.199099999 2743585....,1.253023e+06,2.743585e+06,"(47.41235524185776, 9.34131139316607)","Lerchenfeldstrasse 5, 9014 St.Gallen, Switzerland",Lerchenfeldstrasse 5,9014 St.Gallen
60,9000,St.Gallen,MULTILINESTRING Z ((1252751.217300002 2743899....,Laermemission.10332,10332,GS,62,51,900.0,51.0,...,9014 St.Gallen,9014,St.Gallen,MULTILINESTRING Z ((1252751.217300002 2743899....,1.252751e+06,2.743899e+06,"(47.409841172441034, 9.345382009064819)","Oberstrasse 243, 9014 St.Gallen, Switzerland",Oberstrasse 243,9014 St.Gallen
73,9000,St.Gallen,MULTILINESTRING Z ((1256065.783100002 2749304....,Laermemission.9338,9338,KS,76,67,7800.0,448.0,...,9016 St.Gallen,9016,St.Gallen,MULTILINESTRING Z ((1256065.783100002 2749304....,1.256066e+06,2.749304e+06,"(47.438439517735475, 9.418081911694276)","Schachenstrasse 1, 9016 St.Gallen, Switzerland",Schachenstrasse 1,9016 St.Gallen
78,9000,St.Gallen,MULTILINESTRING Z ((1255502.384199999 2747032....,Laermemission.9126,9126,KS,77,68,11200.0,632.0,...,9008 St.Gallen,9008,St.Gallen,MULTILINESTRING Z ((1255502.384199999 2747032....,1.255502e+06,2.747033e+06,"(47.433885402545165, 9.38779240393882)","Splügenstrasse 15A, 9008 St.Gallen, Switzerland",Splügenstrasse 15A,9008 St.Gallen


Im df ('duplicates_with addressesBEARB) musste ich einige Koordinaten (circa 20) von Hand nachschlagen, weil Google die Adresse nicht zuordnen konnte.

Ausserdem habe ich mit "Suchen und Ersetzen" alle Strassennamen korrigiert, die ich vorher teils zur Bearbeitung falsch gelassen hatte. Z.B. Zürcherstrasse statt Zürcher Strasse.

In [9]:
df_sg = pd.read_csv('daten/df_duplicates_with addresses-BEARB.csv')

In [10]:
df_sg[df_sg['strasse'].duplicated()]

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname


In diesem df ('without duplicates_newBEARB) musste ich einige Koordinaten (circa 10) von Hand nachschlagen, weil Google die Adresse nicht zuordnen konnte. Ausserdem habe ich gleich noch mit "Suchen und Ersetzen" alle Strassennamen korrigiert, die ich vorher teils zur Bearbeitung falsch gelassen hatte. Z.B. Zürcherstrasse statt Zürcher Strasse.

In [13]:
df = pd.read_csv('daten/df_sg_without duplicates_newBEARB.csv')

In [14]:
df[df['strasse'].duplicated()].sort_values('strasse')

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname


In [15]:
df_sg_n = df_sg.append(df)

In [16]:
#In der Spalte "strasse" hat es keine Doppelten mehr:
with pd.option_context("display.max_rows", 100):
    display(df_sg_n[df_sg_n['strasse'].duplicated()].sort_values('strasse', ascending = False)) 
    

,GPLZ,gemeinde,geometry,gml_id,OBJECTID,Strasseneigentümer,Emissionswert Tag,Emissionswert Nacht,Tagesverkehr Jahresmittel,Anzahl Fz Tag pro h,...,Verkehrserhebung,Emissionswert__Lre__Tram_Tag__dB_A__,Emissionswert__Lre__Tram_Nacht__dB_A__,Coordinates,gps,addresses,strasse,addressteil,plz,dorfname
289,9000,St.Gallen,MULTILINESTRING Z ((1253969.268300001 2743817....,Laermemission.9574,9574,GS,72,57,2100.0,119.0,...,NaN,NaN,NaN,1253959.293200001 2743819.7588 0,"(47.4207209420824, 9.344717126984364)","Zwyssigstrasse 28, 9000 St.Gallen, Switzerland",Zwyssigstrasse 28,9000 St.Gallen,9000,St.Gallen
129,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,Laermemission.8809,8809,GS,70,55,2800.0,161.0,...,NaN,NaN,NaN,1254548.731899999 2745686.237199999 0)),"(47.425610307449716, 9.36963651212793)","Zwinglistrasse 21, 9000 St.Gallen, Switzerland",Zwinglistrasse 21,9000 St.Gallen,9000,St.Gallen
214,9000,St.Gallen,MULTILINESTRING Z ((1254556.100299999 2745691....,Laermemission.9242,9242,GS,61,49,1000.0,54.0,...,NaN,NaN,NaN,1254551.704100002 2745676.312100001 0,"(47.425639228695424, 9.369505981703966)","Winkelriedstrasse 27, 9000 St.Gallen, Switzerland",Winkelriedstrasse 27,9000 St.Gallen,9000,St.Gallen
130,9000,St.Gallen,MULTILINESTRING Z ((1254548.731899999 2745686....,Laermemission.8810,8810,GS,72,57,2800.0,161.0,...,NaN,NaN,NaN,1254497.153099999 2745648.8475 0)),"(47.42515481911186, 9.369124306625407)","Winkelriedstrasse 20, 9000 St.Gallen, Switzerland",Winkelriedstrasse 20,9000 St.Gallen,9000,St.Gallen
177,9000,St.Gallen,MULTILINESTRING Z ((1254493.754000001 2746970....,Laermemission.9012,9012,KS,74,60,4400.0,253.0,...,NaN,62.9,56.5,1254496.261399999 2746980.327399999 0,"(47.42485077825742, 9.386764845789028)","Wildeggstrasse 46, 9000 St.Gallen, Switzerland",Wildeggstrasse 46,9000 St.Gallen,9000,St.Gallen
128,9000,St.Gallen,MULTILINESTRING Z ((1254445.226100001 2745612....,Laermemission.8808,8808,GS,70,55,2800.0,161.0,...,NaN,NaN,NaN,1254433.771000002 2745604.330499999 0)),"(47.42459477740486, 9.368513832267185)","Tellstrasse 16, 9000 St.Gallen, Switzerland",Tellstrasse 16,9000 St.Gallen,9000,St.Gallen
290,9000,St.Gallen,MULTILINESTRING Z ((1253874.180599999 2743916....,Laermemission.9575,9575,GS,70,55,2100.0,119.0,...,NaN,NaN,NaN,1253858.325100001 2743937.481199998 0,"(47.41978735703927, 9.346244191674261)","Sömmerliwaldstrasse 21, 9000 St.Gallen, Switze...",Sömmerliwaldstrasse 21,9000 St.Gallen,9000,St.Gallen
445,9000,St.Gallen,MULTILINESTRING Z ((1254149.679900002 2745688....,Laermemission.9281,9281,GS,75,68,9100.0,518.0,...,NaN,NaN,NaN,1254152.802900001 2745695.2392 0,"(47.42204836693856, 9.369626628252258)","St.Leonhard-Strasse 45, 9001 St.Gallen, Switze...",St.Leonhard-Strasse 45,9001 St.Gallen,9001,St.Gallen
226,9000,St.Gallen,MULTILINESTRING Z ((1254150.7744 2744280.61300...,Laermemission.9265,9265,KS,76,64,7000.0,404.0,...,NaN,NaN,NaN,1254160.658199999 2744316.5482 0)),"(47.42242275855416, 9.35136369428095)","St.Josefen-Strasse 7, 9000 St.Gallen, Switzerland",St.Josefen-Strasse 7,9000 St.Gallen,9000,St.Gallen
232,9000,St.Gallen,MULTILINESTRING Z ((1254149.4243 2743627.16030...,Laermemission.9274,9274,KS,77,65,6900.0,399.0,...,NaN,NaN,NaN,1254135.607099999 2743651.182300001 0,"(47.4223430770924, 9.342540507020162)","St.Josefen-Strasse 50, 9000 St.Gallen, Switzer...",St.Josefen-Strasse 50,9000 St.Gallen,9000,St.Gallen


In [17]:
df_sg_n.to_csv('daten/Ergebnisse/df_sg_n_FINAL.csv', index = False)

## Rest der Auswertungen: nächstes Notebook